<h1>Q3.3 Gibbs Sampling</h1>

In [13]:
import numpy as np
from math import exp
import pandas as pd

class Gibbs:
    def __init__(self, beta : float, num_iter : iter = 1000000):
        self.beta = beta
        self.num_iter = num_iter
        self.x = np.random.choice([0,1], size=(10,10))
        self.initial_x = self.x.copy()

    def run_sampling(self):
        self.results = []
        
        for _ in range(self.num_iter):
            x_ij = np.random.choice(10, size=(2,))
            i, j = x_ij[0], x_ij[1]
            
            s_1, s_0 = 0, 0
            (s_1, s_0) = (s_1, s_0) if i == 0 else (s_1 + (self.x[i-1][j] == 1), s_0 + (self.x[i-1][j] == 0))
            (s_1, s_0) = (s_1, s_0) if i == 9 else (s_1 + (self.x[i+1][j] == 1), s_0 + (self.x[i+1][j] == 0))
            (s_1, s_0) = (s_1, s_0) if j == 0 else (s_1 + (self.x[i][j-1] == 1), s_0 + (self.x[i][j-1] == 0))
            (s_1, s_0) = (s_1, s_0) if j == 9 else (s_1 + (self.x[i][j+1] == 1), s_0 + (self.x[i][j+1] == 0))
            # print(s_1, s_0)
            
            e_s_1 = exp(self.beta * s_1)
            e_s_0 = exp(self.beta * s_0)
            
            p_1 = e_s_1 / (e_s_1 + e_s_0)
            # print(p_1)
            
            new_x_ij = np.random.choice([0,1], p=[1-p_1, p_1])
            # print(f"x[{i}][{j}]: {x[i][j]} => {new_x_ij}")
        
            self.x[i][j] = new_x_ij
            
            self.results.append((i, j, new_x_ij))

    def compute_joint(self):
        prob_table = np.zeros((2,2))
        
        x_1_10, x_10_10 = self.initial_x[0][9], self.initial_x[9][9]
        
        for r in self.results:
            if r[1] == 9:
                if r[0] == 0:
                    x_1_10 = r[2]
                elif r[0] == 9:
                    x_10_10 = r[2]
            prob_table[x_1_10][x_10_10] += 1
        
        self.prob_table = pd.DataFrame(prob_table / np.sum(prob_table))

        print(self.prob_table)

    def run(self):
        self.run_sampling()
        self.compute_joint()

In [15]:
gibbs_4 = Gibbs(4)
gibbs_1 = Gibbs(1)
gibbs_001 = Gibbs(0.01)

In [17]:
gibbs_4.run()

          0         1
0  0.997739  0.001487
1  0.000756  0.000018


In [18]:
gibbs_1.run()

          0         1
0  0.354386  0.232140
1  0.205180  0.208294


In [19]:
gibbs_001.run()

          0         1
0  0.257205  0.250378
1  0.245429  0.246988
